In [1]:
# Import libraries
import gc
import math
import pandas as pd
import numpy as np
import gensim.models as gsm
import multiprocessing as mp
import datetime as dt

from scipy.spatial.distance import cosine

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import naive_bayes
from sklearn.metrics import roc_auc_score

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import model_selection
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

C:\Users\CKhan\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\CKhan\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%reset Out
gc.collect()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (0 entries)


0

### Train-Test Split

In [3]:
X_train = pd.read_pickle('./pickles.gi/X_train_tfidf_doc2vec_cossim.pkl')
X_test = pd.read_pickle('./pickles.gi/X_test_tfidf_doc2vec_cossim.pkl')

In [19]:
train_lookup_df = pd.read_pickle('./pickles.gi/train_lookup_df.pkl')
y_train = train_lookup_df.is_duplicate

In [ ]:
%reset_selective train_lookup_df
gc.collect()

### Evaluation Metrics

In [9]:
def kfoldScore(results):
    mean_score = results.mean()
    std_dev = results.std()
    std_error = results.std() / math.sqrt(results.shape[0])
    ci =  2.262 * std_error
    lower_bound = mean_score - ci
    upper_bound = mean_score + ci
    print ("Score is %f +/-  %f" % (mean_score, ci))
    print ('95 percent probability that if this experiment were repeated over and over the average score would be between %f and %f' % (lower_bound, upper_bound))

### Common Setup

In [10]:
seed = 1
kfold = model_selection.KFold(n_splits=10, random_state=seed)
neg_log_loss = 'neg_log_loss'
roc_auc = 'roc_auc'

### Logistic Regression

In [11]:
# Baseline logistic regression model
logreg_model_base = LogisticRegression()

In [14]:
# Kfold Cross Validation: ROC AUC
logreg_model_base_rocauc = model_selection.cross_val_score(logreg_model_base, X_train, y_train, cv=kfold, scoring=roc_auc)
print('ROC AUC')
kfoldScore(logreg_model_base_rocauc)

ROC AUC
Score is 0.500780 +/-  0.003018
95 percent probability that if this experiment were repeated over and over the average score would be between 0.497762 and 0.503798


In [15]:
# Kfold Cross Validation: NEG LOG LOSS
logreg_model_base_logloss = model_selection.cross_val_score(logreg_model_base, X_train, y_train, cv=kfold, scoring=neg_log_loss)
print('NEG LOG LOSS')
kfoldScore(logreg_model_base_rocauc)

NEG LOG LOSS
Score is 0.500780 +/-  0.003018
95 percent probability that if this experiment were repeated over and over the average score would be between 0.497762 and 0.503798


In [16]:
# Predict Outcome Probabilities
logreg_model_base.fit(X_train, y_train)
y_pred_logreg_model_base = logreg_model_base.predict_proba(X_test)

In [24]:
# Create submission CSV
submission = X_test.pid.to_frame()
submission['is_duplicate'] =  pd.Series([row[1] for row in y_pred_logreg_model_base])
submission.columns = ['test_id', 'is_duplicate']
submission.to_csv('./submissions.gi/submission02.csv', index=False)